# Stage 0: SETUP
The following libraries are used directly. For the full list of isntalled ppackages and versions, please see requuirements.txt

In [1]:
# For accessing ORES API
import requests

# For processing
import pandas as pd
import numpy as np


# Stage 1: Data Acquisition

Data is downloaded as csv files, and is already available in this repository in the data folder. See the readme for details on the source of the data.

## Page Data
Page data is downloaded from [this](https://figshare.com/articles/dataset/Untitled_Item/5513449) repository.

In [19]:
page_data = pd.read_csv('data/raw/page_data.csv')
page_data.head()

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070
4,Template:Namibia-politician-stub,Namibia,391862409


## Population Data
Population Data is downloaded from [this](https://docs.google.com/spreadsheets/d/1CFJO2zna2No5KqNm9rPK5PCACoXKzb-nycJFhV689Iw/edit#gid=283125346) google doc.

In [3]:
pop_data = pd.read_csv('data/raw/WPDS_2020_data.csv')
pop_data.head()

,FIPS,Name,Type,TimeFrame,Data (M),Population
0,WORLD,WORLD,World,2019,7772.850,7772850000
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000
3,DZ,Algeria,Country,2019,44.357,44357000
4,EG,Egypt,Country,2019,100.803,100803000


# Stage 2: Data Processing
In this stage we combine and clean the data, and use the [ORES](https://github.com/wikimedia/ores) client to get predicted article quality.

# Clean and combine
Remove the templates from page data and regions from pop data

In [22]:
page_data_clean = page_data.loc[~page_data["page"].str.contains("^Template"), :]
pop_data_clean = pop_data.loc[pop_data["Type"] == "Country", :]

# Left join to keep countries without articles. Keep revid as int
combined_data = pop_data_clean.merge(page_data_clean, how="outer", left_on="Name", right_on="country")

# Record unmatched countries
unmatched_pop_data = combined_data[combined_data["page"].isna()].drop(columns=page_data_clean.columns)
unmatched_pop_data.to_csv('data/unmatched/wp_wpds_countries-no_match.csv')
print("{} Countries could not be matched".format(len(unmatched_pop_data)))

# Record unmatched pages
unmatched_page_data = combined_data[combined_data["Name"].isna()].drop(columns=pop_data_clean.columns)
unmatched_page_data.to_csv('data/unmatched/page_data-no_match.csv')
print("{} Pages could not be matched".format(len(unmatched_page_data)))

# Clean
combined_data_complete = combined_data.dropna()
combined_data_complete["rev_id"] = combined_data_complete["rev_id"].astype(int)

26 Countries could not be matched
1859 Pages could not be matched


C:\Users\ccase\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Get ORES Data
Data is acquired from the [ORES API](https://ores.wikimedia.org/v3/#). We request the "articlequality" model from the "enwiki" context for batches of revids at a time. Max 50 per request.

In [23]:
# Takes a batch of revids
def api_call(revids, context='enwiki', model='articlequality'):
    endpoint = "https://ores.wikimedia.org/v3/scores/{context}".format(context=context)
    headers = {
        'User-Agent': 'https://github.com/TheCaseca',
        'From': 'ccase20@uw.edu'
    }
    call = requests.get(endpoint, headers=headers, params={"models":model, "revids": "|".join(revids)})
    response = call.json()
    
    return response

In [36]:
# Index by revids for easier updating
revids = combined_data_complete["rev_id"]
combined_data_complete.set_index('rev_id')

revids = revids.astype(int)
revids = revids.astype(str)

n = 50
preds = {}
for i in range(len(revids)//n):
    if i % 100 == 0:
        print("Collecting rows {} to {} of {}".format(i*n, (i*100+1)*n, len(revids)))
    api_data = api_call(list(revids[i*n: (i+1)*n]))
    new_preds = {revid: score['articlequality'].get('score', {}).get('prediction') for revid, score in api_data['enwiki']['scores'].items()}
    preds.update(new_preds)

# Update dataframe
pred_df = pd.DataFrame.from_dict(preds, orient='index', columns=['prediction'])
pred_df.index = pred_df.index.astype(int)
combined_data_complete_preds = combined_data_complete.merge(pred_df, left_on='rev_id', right_index=True)
combined_data_complete_preds.head()

,FIPS,Name,Type,TimeFrame,Data (M),Population,page,country,rev_id
0,DZ,Algeria,Country,2019.0,44.357,44357000.0,Ali Fawzi Rebaine,Algeria,686269631
1,DZ,Algeria,Country,2019.0,44.357,44357000.0,Ahmed Attaf,Algeria,705910185
2,DZ,Algeria,Country,2019.0,44.357,44357000.0,Ahmed Djoghlaf,Algeria,707427823
3,DZ,Algeria,Country,2019.0,44.357,44357000.0,Hammi Larouissi,Algeria,708060571
4,DZ,Algeria,Country,2019.0,44.357,44357000.0,Salah Goudjil,Algeria,708980561


We filter out any missing predictions and record them. 274 articles did not have a prediction from ORES.

In [51]:
# Recordd missing
missing_preds = combined_data_complete_preds[combined_data_complete_preds['prediction'].isna()]
missing_preds.to_csv('data/unmatched/wp_wpds_politicians-no_prediction.csv')
print("{} Pages could not be predicted".format(len(missing_preds)))

# Remove from data
final_data = combined_data_complete_preds.dropna(subset=['prediction'])

274 Pages could not be predicted


In [50]:
final_data

,FIPS,Name,Type,TimeFrame,Data (M),Population,page,country,rev_id,prediction
0,DZ,Algeria,Country,2019.0,44.357,44357000.0,Ali Fawzi Rebaine,Algeria,686269631,Stub
1,DZ,Algeria,Country,2019.0,44.357,44357000.0,Ahmed Attaf,Algeria,705910185,Stub
2,DZ,Algeria,Country,2019.0,44.357,44357000.0,Ahmed Djoghlaf,Algeria,707427823,Stub
3,DZ,Algeria,Country,2019.0,44.357,44357000.0,Hammi Larouissi,Algeria,708060571,Stub
4,DZ,Algeria,Country,2019.0,44.357,44357000.0,Salah Goudjil,Algeria,708980561,Stub
...,...,...,...,...,...,...,...,...,...,...
44833,VU,Vanuatu,Country,2019.0,0.321,321000.0,Ralph Regenvanu,Vanuatu,787719834,B
44834,VU,Vanuatu,Country,2019.0,0.321,321000.0,Sethy Regenvanu,Vanuatu,788200153,Stub
44835,VU,Vanuatu,Country,2019.0,0.321,321000.0,Ati George Sokomanu,Vanuatu,788303161,Stub
44836,VU,Vanuatu,Country,2019.0,0.321,321000.0,Antoine Wright (politician),Vanuatu,789446881,Start
